Python_Data_Science_2(Text Mining)

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
import numpy as np
#from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [25]:
data = pd.read_csv('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/spambase.data', engine='python', header=None)

In [26]:
data = data.sample(frac = 1).reset_index(drop = True)
Xtrain = data.iloc[:-100,:-1]
Ytrain = data.iloc[:-100,-1]

Xtest = data.iloc[-100:,:-1]
Ytest = data.iloc[-100:,-1]

data = pd.read_csv('spambase.data').values # use pandas for convenience # gives a matrix
np.random.shuffle(data) #inplace shuffling
or
data = pd.read_csv('spambase.data').as_matrix() # use pandas for convenience # gives a matrix

In [27]:
# Naive Bayes
model = MultinomialNB()
model.fit(Xtrain, Ytrain)
print("Classification rate for NB:", model.score(Xtest, Ytest))

Classification rate for NB: 0.7


In [28]:
# AdaBoost
model2 = AdaBoostClassifier()
model.fit(Xtrain, Ytrain)
print("Classification rate for Adaboost:", model.score(Xtest, Ytest))

Classification rate for Adaboost: 0.7


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

In [30]:
spam_df = pd.read_csv('C:/Users/rpothams/Downloads/BD/NLP/spam.csv', encoding= 'iso-8859-1')
spam_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [31]:
spam_df = spam_df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
spam_df.columns = ['labels', 'data']

In [32]:
#creating binary labels
spam_df['b_labels'] = spam_df['labels'].map({'ham': 0, 'spam': 1})
Y = spam_df['b_labels'].values #numpy array

In [33]:
sentences = spam_df['data']
# using CountVectorizer
count_vect = CountVectorizer(decode_error='ignore')
X_count = count_vect.fit_transform(sentences)

In [34]:
tfidf = TfidfVectorizer()
# fit and then transform
tfidf.fit(sentences)
X_tfidf = tfidf.transform(sentences) #gives a numpy sparse matrix
#both fit and transform in one step
#X = tfidf.fit_transform(sentences)

In [35]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_tfidf, Y, test_size = 0.33)
Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X_count, Y, test_size = 0.33)

In [36]:
print(Xtrain2[1])

  (0, 5223)	1
  (0, 5525)	1
  (0, 8284)	1
  (0, 4058)	2
  (0, 4522)	1
  (0, 7561)	1


In [37]:
model_countV = MultinomialNB()
model_countV.fit(Xtrain2, Ytrain2)
print("train score:", model_countV.score(Xtrain2, Ytrain))
print("test score:", model_countV.score(Xtest2, Ytest))

train score: 0.7656040717921243
test score: 0.7678085916258837


In [38]:
model_tfidf = MultinomialNB()
model_tfidf.fit(Xtrain, Ytrain)
print("train score:", model_tfidf.score(Xtrain, Ytrain))
print("test score:", model_tfidf.score(Xtest, Ytest))

train score: 0.9619608893651219
test score: 0.9559543230016313


In [ ]:
# wordcloud
from wordcloud import WordCloud
def visualize(df, label):
    words = ''
    for msg in df[df.iloc[:,1] == label].iloc[:,0]:
        msg = msg.lower()
        words += msg + ' '
    wordcloud = WordCloud(width=600, height=400).generate(words)
    plt.imshow(worcloud) #draws an image on the current figure (creating a figure is there isn't a current figure)
    plt.show()

In [ ]:
-*-**

#Section 3 - Psentiment Analysis
XML Parser - BeautifulSoup

#### Tokenization
why not just use .split for tokenization?
    contractions - combination of multiple words
        don't = do not
        won't = would not
        she'll = she will
    Extract punctuations and symbols: Period (.) and Dollar ($)
    This is all done by nltk.tokenize.word_tokenize

Custom Tokenizer (to perform better than in built nltk)
    - remove stopwords
    - remove numbers and single-letter tokens
    - application specific
    
#### Tokens into Vectors
    - scikitLear has it inbuilt
    - lets write it manually
    - each word is a key and index is a value - Dictionary/map
    

In [ ]:
# if the word is present, turn the value to 1, irresepective of frequency
V = 2000 #Vocab size
x = np.zeros(V)
for token in sentence:
    index = word_index_map[token]
    x[index] = 1
    #x[index] += 1 # for raw counts

In [121]:
# For sentiment analysis, we'll use raw counts, but qhat's the problem?
# Really long docs become really long vectors, really short docs have short vectors (ex: amazon long and short reviews)
# classifier have to deal with vectors of varying lengths and we dont want comparison based on length of vectors, hnce normalize
# Vector/# of words in document

# Another reason for normalization
# LogisticRegression performs better with normalizaed data (sigmoid(10) ~= sigmoid(100) ~= 1)

import nltk
from nltk.stem import WordNetLemmatizer # turns words in to base forms i.e dogs -> dog
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

wordnet_lemmatizer = WordNetLemmatizer()
stopwords_cust = set(w.rstrip() for w in open('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/stopwords.txt'))

# positive.review is a XML file with Amazon positive reviews
positive_reviews = BeautifulSoup(open('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/electronics/positive.review').read(), 'lxml')
positive_reviews = positive_reviews.findAll('review_text')
negative_reviews = BeautifulSoup(open('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/electronics/negative.review').read(), 'lxml')
negative_reviews = negative_reviews.findAll('review_text')

np.random.shuffle(positive_reviews)

In [86]:
positive_reviews[0].text

'\nThere\'s no need to take anybody else\'s word for specs on this Sony DVD player when you can view or download, for free, the user\'s manual at http://esupport.sony.com/US/perl/model-documents.pl?mdl=DVPNS90V&LOC=3. I\'ve been running this Sony DVP-NS90V player through a Sony widescreen HDTV and a Pioneer amplifier/receiver for several days now, and I find almost none of the problems and limitations mentioned by previous reviewers. One, for example, states that there is "no bass management," yet you can (a) route more or less signal to the subwoofer by selecting "small" or "large" for the front speaker size (see page 76 of the user\'s manual), and also (b) increase or decrease the bass from -15 dB to +5 dB (see page 77 of the manual). Whether you prefer analog to digital audio, or you think SACD\'s are tops, is a matter of opinion, taste and choice and not an objective evaluation of a playback unit. I, for one, prefer DVD-Audio, and I mention this for an important reason: Some produc

In [87]:
nltk.tokenize.word_tokenize(positive_reviews[0].text.lower())

['there',
 "'s",
 'no',
 'need',
 'to',
 'take',
 'anybody',
 'else',
 "'s",
 'word',
 'for',
 'specs',
 'on',
 'this',
 'sony',
 'dvd',
 'player',
 'when',
 'you',
 'can',
 'view',
 'or',
 'download',
 ',',
 'for',
 'free',
 ',',
 'the',
 'user',
 "'s",
 'manual',
 'at',
 'http',
 ':',
 '//esupport.sony.com/us/perl/model-documents.pl',
 '?',
 'mdl=dvpns90v',
 '&',
 'loc=3',
 '.',
 'i',
 "'ve",
 'been',
 'running',
 'this',
 'sony',
 'dvp-ns90v',
 'player',
 'through',
 'a',
 'sony',
 'widescreen',
 'hdtv',
 'and',
 'a',
 'pioneer',
 'amplifier/receiver',
 'for',
 'several',
 'days',
 'now',
 ',',
 'and',
 'i',
 'find',
 'almost',
 'none',
 'of',
 'the',
 'problems',
 'and',
 'limitations',
 'mentioned',
 'by',
 'previous',
 'reviewers',
 '.',
 'one',
 ',',
 'for',
 'example',
 ',',
 'states',
 'that',
 'there',
 'is',
 '``',
 'no',
 'bass',
 'management',
 ',',
 "''",
 'yet',
 'you',
 'can',
 '(',
 'a',
 ')',
 'route',
 'more',
 'or',
 'less',
 'signal',
 'to',
 'the',
 'subwoofer',
 

In [126]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rpothams\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [143]:
def my_tokenizer(s):
    temp = []
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [char for char in tokens if char not in string.punctuation]
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stopwords_cust] #(using a custom stopwords in stopwords.txt)
    #tokens = [char for char in tokens if char not in stop_words]
    return tokens

word_index_map = {}
current_index = 0
positive_tokenized = []
negative_tokenized = []

for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    print(tokens)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
            
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

["'ve", 'similar', 'cat', 'occasionally', 'knocking', 'walking', "'ve", 'crack', 'hold', 'disc', 'securely', 'force', 'you', 'risk', 'damaging', 'disc', 'removing', 'you', 'properly', 'front', 'pulling', 'near', 'corner', 'holding', 'colored', 'fine']
['this', 'amplifier', 'stated', 'increased', 'modem', 'downstream', 'obvious', 'loss', 'noise', 'ratio', 'upstream', 'signal', 'channel', 'sharper', 'cable', 'amplifier', 'rg59', 'cable', 'line', 'minimum', 'loss', 'broadband', 'signal', 'suggest', 'you', 'replace', 'cable', 'carry', 'cable', 'signal', 'rg59', 'power', 'supply']
['handy', 'little', 'sticker', 'you', 'write', 'contact', 'info', 'you', 'lose', 'slightly', 'costl']
['bought', 'this', 'screen', 'replacement', '8ms', 'hyundai', 'l90d+', 'low-priced', 'screen', 'gaming', 'wa', 'replacement', 'dell', 'fp2100', 'wife', 'anyways', 'this', 'excellent', 'monitor', 'excellent', 'color', 'reproduction', 'response', 'time', 'flight', 'sims', 'movie', 'beautiful', 'weird', 'resolution',

['bought', 'mine', 'week', 'ago', "n't", 'reason', 'star', 'instead', 'wa', 'little', 'hard', 'plug', 'then', '100', 'pleased', 'quality', 'close', 'buy']
['cable', 'super', 'dvd', 'player', 'absolutely', 'interference', 'home', 'theater', 'equipment', 'picture', 'soo', 'vivid', 'cable', 'themselves', 'hold', '100', 'pound', 'weight', 'doubt', 'you', 'monster', 'cable', 'you', "n't", 'sorry']
['researched', 'lot', 'bought', 'this', 'money', 'this', 'wa', 'hand', 'lcd', 'amazon', 'price', 'shipped', 'house', 'timely', 'manner', 'major', 'issue', 'mainly', 'this', 'xbox', '360.', 'you', 'disappointed', 'you', 'planning', 'doing', 'game', 'mode', 'doe', 'enhance', 'picture', 'deal', 'provides', 'sort', 'anti', 'aliasing', 'effect', 'smoothing', 'rough', 'edge', 'owning', 'month', "n't", 'happier', 'poor', 'speaker', 'plug', 'nice', 'set', 'headphone', 'game', 'sound', "'ve", 'hooked', 'antenna', 'regular', 'cable', 'channel', 'picture', 'air', 'football', 'game', 'look', 'this', 'samsung'

['this', 'cartridge', 'purchase', 'elsewhere', 'cheaper', 'printer', 'issue', 'whatsoever', 'printer', 'print', 'fantastic', 'photo', 'ink', 'premium', 'photo', 'paper']
['epson', 'photo', '1270', 'photo', 'quality', 'color', 'excellent', 'printed', 'digital', 'photo', 'people', 'believe', 'inkjet', 'printer']
['purchased', 'this', 'wife', 'pavilion', 'notebook', 'dv5139us', 'screen', 'size', '15.4', '1.8', 'wdth', '14.1', 'depth', '10.4', 'bag', 'personal', 'business', 'moderate', 'extra', 'compartment', 'organization', 'this', 'ha', 'met', 'light', 'weight', 'constructed', 'durable', 'material', 'organized', 'notebook', 'fit', 'this', 'top', 'loading', 'ease', 'satisfied', 'this', 'purchase', 'recommend', 'purchasing', 'this']
['you', 'buy', 'this', 'service', 'plan', 'print', 'contract', 'receipt', 'purchase', 'plan', 'looked', 'contract', 'intensely', 'nearly', 'note', 'contract', 'united', 'including', 'washington', 'minus', 'maine', 'doe', 'include', 'guam', 'virgin', 'island', '

['this', 'little', 'piece', 'hardware', 'bought', 'mp3', 'movie', 'pleased', 'totally', 'plug-n-play', "n't", 'driver', 'updating', 'assuming', 'you', 'par', '160gb', '149', 'actuality', 'space', 'moving', 'nearly', 'gig', 'day', 'froze', 'minor', 'incident', "'ve", 'machine', 'run', 'alot', 'faster', '100g', 'music', 'happy', 'ha', 'fuss', 'doe', "n't", 'noise', 'silent', 'fart', 'plus', 'found', 'mine', 'wal-mart', '114', 'purchase', 'you', "'re", 'budget', 'this', 'seagate', '300gb', 'highly', 'recommended', "'ve"]
['price', 'fine', 'plastic', 'solid', 'ha', 'feeling', 'low', 'quality']
['this', 'wa', 'cheapest', 'mount', 'option', 'expensive', 'pull', 'wall', 'allow', 'screen', 'pivot', 'down.it', 'compact', 'close', 'wall', 'dont', 'feature', 'mount', 'smooth', 'move', 'deflection', 'sloppiness', 'construction', 'altough', 'lcd']
['product-wise', 'card', 'ha', 'fine', 'then', 'memory', 'card', "n't", 'product', 'wa', 'shipped', 'timely', 'manner', 'price', 'wa', 'reasonable']
['me

['happy', 'this', 'model', 'vcr', 'replacement', 'record', 'broadcast', 'you', "n't", 'beat', 'price', 'this', 'feature-set', "'ll", 'try', 'describe', 'aspect', "n't", 'mentioned', 'review', 'owner', 'manual', 'found', 'online', 'pdf', 'format', 'schedule', 'upto', 'program', 'doe', 'support', 'vcr+', 'tvio-style', 'program', 'guide', 'you', 'enter', 'title', 'schedule', 'appears', 'schedule', 'list', 'direct', 'navigator', 'chapter', 'menu', 'sort', 'main', 'schedule', 'listing', 'indicates', 'event', 'current', 'disk', 'on-screen', 'menu', 'you', 'remote', 'control', 'button', 'functional', 'screen', 'commercial', 'skip', 'button', 'advance', "n't", 'granular', 'click', 'little', 'fast', 'forwarding', 'remote', 'operation', 'fast', 'feeling', 'response', 'compared', 'tape', 'player', 'complaint', 'remote', 'sensor', 'recorder', 'right-hand', 'unit', 'video', 'cabinet', 'sensor', 'deadzone', 'half', "n't", 'shift', 'position', 'disk', 'tray', 'slot', 'left', "n't", 'close', 'wall', '

['talking', 'mp3', 'player', 'ipod', 'heard', 'you', 'add', 'file', 'window', 'format', 'music', 'received', 'this', 'e260', 'mp3', 'player', 'birthday', 'june', '2006', 'this', 'mp3', 'player', 'owned', 'little', 'quirk', 'overall', 'disappointed', 'recommend', 'this', 'mp3', 'player', 'rechargeable', 'battery-powered', 'item', 'required', 'lengthy', 'initial', 'charge', 'future', 'charge', "n't", 'mp3', 'player', 'play', 'hour', 'charge', "'ve", 'run', 'completely', 'continuously', 'hour', 'battery', 'issue', 'installed', 'software', 'computer', 'disk', 'you', 'owner', 'manual', 'wa', 'bit', 'irritated', 'wa', 'hard', 'copy', 'owner', 'manual', 'week', 'ago', 'issue', 'mp3', 'player', 'froze', 'computer', 'look', 'troubleshooting', 'guide', 'player', 'freeze', 'you', 'hold', 'power', 'button', 'reset', 'pre-loaded', 'song', 'heard', 'wa', 'no-name', 'band', 'played', 'saturday', 'night', 'live', 'instinct', 'wa', 'delete', 'song', 'own', 'choice', 'decided', 'leave', 'space', 'song',

['this', 'little', 'charger', 'purchased', 'lot', 'abuse', 'replace', 'buy', 'charge', 'quickly', 'handy']
['little', 'trouble', 'getting', 'system', 'recognize', 'set', '...', 'little', 'tweaking', 'beautifully', 'previously', 'owned', 'regular', 'crt', '...', 'improvement', 'superb', 'clarity', 'brightness', 'money', 'you', "n't", 'wrong', 'this']
['microsoft', 'optical', 'wheel', 'mouse', 'product', 'worth', 'penny', 'purchase', 'price', 'value', 'mouse', 'ball', 'optical', 'wheel', 'mouse', 'accurate', 'purchase']
['simple', 'device', 'doe', 'job', 'you', 'note', 'led', 'device', 'remain', 'lit', 'device', 'plugged', 'powered', 'glow', 'bright', 'blue']
['this', 'drive', 'sucessfully', 'using', 'backup', 'mac', 'recomend', 'line', 'storag']
['this', 'charger', 'designed', 'ipod', 'except', 'shuffle', 'shuffle', 'connects', 'via', 'usb', 'this', 'charger', 'firewire', 'ipod', 'charge', 'connected', 'computer', 'via', 'firewire', 'usb', '2.0', 'this', 'charger', 'allows', 'you', 'cha

['this', 'transmitter', 'bomb', 'diggiti', 'pro', 'clarity', 'ha', 'neck', 'pro', 'you', 'own', 'chyrsler', 'cruiser', 'you', 'love', 'this', 'device', 'neck', 'brings', 'ipod', 'top', 'steering', 'column', 'view', 'you', "n't", 'look', 'fumble', 'you', 'i-pod', 'seat', 'pro', 'stable', 'outlet', 'pro', 'reception', 'con', 'nice', 'on/off', 'switch', 'base', 'outlet', 'stay', 'car', 'green', 'light', 'device', 'attract', 'unwanted', 'attention', 'con', 'protector', 'everytime', 'ipod', 'car']
['amazon', 'partner', '.01', 'plus', 'shipping', 'then', 'discount', 'price', 'wa', 'lower', 'le', 'shipping', 'cost', 'true', 'seller', 'little', 'shipping', 'maybe', '4.50', 'counting', 'time', 'apple', 'packaging', "n't", 'mention', 'nano', 'bit', 'trepidation', 'plugged', 'wall', 'ipod', 'nano', 'liked', 'charging', 'happened', 'nano', 'reviewer', 'buy', 'this', 'avoid', 'price', 'elsewhere']
['...', 'ha', 'xr55', 'doe', 'hdmi', 'dual-amping', 'front', 'main', 'available', '5.1', 'playback', '

In [144]:
# we will use proportions in vectors
def tokens_to_vectors(tokens, label):
    data_vect = np.zeros(len(word_index_map) + 1) # +1 for label
    for t in tokens:
        index = word_index_map[t]
        data_vect[index] += 1
    data_vect = data_vect/data_vect.sum()
    data_vect[-1] = label
    return data_vect

N = len(positive_tokenized) + len(negative_tokenized)
data = np.zeros((N, len(word_index_map) + 1))
i = 0

for tokens in positive_tokenized:
    xy = tokens_to_vectors(tokens, 1)
    data[i,:] = xy
    i += 1

for tokens in negative_tokenized:
    xy = tokens_to_vectors(tokens, 0)
    data[i,:] = xy
    i += 1
    
np.random.shuffle(data)

In [145]:
X = data[:, :-1]
Y = data[:, -1]

Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print("Classification rate: ", model.score(Xtest, Ytest))

Classification rate:  0.78


In [146]:
# now by looking at weights lets see if the word has positive or negative sentiment
# instead of looking at all weights, lets look at values far from 0

threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

've 0.6862625106973683
you 1.0263095402432902
cable 0.8031732957445152
little 0.8819971845120054
wa -1.4442459931421412
excellent 1.3143974232243403
time -0.6268309232226386
ha 0.7229246854899519
n't -1.9703064914317787
week -0.6554836632196015
easy 1.8505556985271365
picture 0.5488211617017572
speaker 0.9877465383044356
month -0.7577298279964364
love 1.1864579794363446
sound 1.0607419433054655
doe -1.1488204658034291
bit 0.6115197545416682
quality 1.4816845597123192
price 2.4520743748245275
then -1.0581305214570595
money -1.046851736258084
memory 0.975322498078638
company -0.5395469296903646
expected 0.5747613739599412
tried -0.8165394244258741
try -0.6523049865497847
card -0.5227353047552664
buy -0.7956977096063363
look 0.5120479975664596
highly 1.0139788982788358
recommend 0.7162835355380468
using 0.5700218884190628
space 0.6038627672319884
pretty 0.77020090542522
useless -0.5005660759406584
unit -0.5814693448069288
sent -0.5014504416497959
item -0.9032923196890771
returned -0.81229

In [147]:
#Note: So far we have not handled contractions n't --> not, I've --> I have